In [ ]:
from google.colab import drive
google.drive.mount('/content/drive')

In [2]:
!mkdir -p /content/
!mkdir -p /content/KITTI_STEP
!mkdir -p /content/KITTI_STEP/frames_split
!mkdir -p /content/KITTI_STEP/annotations_split

In [3]:
!mkdir -p /content/KITTI_STEP/frames
!mkdir -p /content/KITTI_STEP/annotations

In [4]:
#Add Path Where Zips are Stored
!unzip -q /content/drive/MyDrive/data_tracking_image_2.zip -d /content/KITTI_STEP/frames_split
!tar -xvzf /content/drive/MyDrive/kitti-step.tar.gz -C /content/KITTI_STEP/annotations_split

Streaming output truncated to the last 5000 lines.
kitti-step/panoptic_maps/train/0017/000086.png
kitti-step/panoptic_maps/train/0017/000031.png
kitti-step/panoptic_maps/train/0017/000116.png
kitti-step/panoptic_maps/train/0017/000094.png
kitti-step/panoptic_maps/train/0017/000136.png
kitti-step/panoptic_maps/train/0017/000027.png
kitti-step/panoptic_maps/train/0017/000025.png
kitti-step/panoptic_maps/train/0017/000036.png
kitti-step/panoptic_maps/train/0017/000113.png
kitti-step/panoptic_maps/train/0017/000080.png
kitti-step/panoptic_maps/train/0017/000063.png
kitti-step/panoptic_maps/train/0017/000122.png
kitti-step/panoptic_maps/train/0017/000107.png
kitti-step/panoptic_maps/train/0017/000059.png
kitti-step/panoptic_maps/train/0017/000026.png
kitti-step/panoptic_maps/train/0017/000056.png
kitti-step/panoptic_maps/train/0017/000087.png
kitti-step/panoptic_maps/train/0017/000023.png
kitti-step/panoptic_maps/train/0017/000138.png
kitti-step/panoptic_maps/train/0017/000143.png
kitti-ste

In [5]:
import os
import shutil

def copy_and_rename_recursive(source_root, dest_folder):
    """
    Recursively traverse the source_root directory and copy each .png file
    into dest_folder. The new filename is prefixed with the relative path,
    using underscores for subdirectory separation.
    """
    for dirpath, _, filenames in os.walk(source_root):
        # Compute the relative subdirectory path from the source_root
        rel_path = os.path.relpath(dirpath, source_root)
        # If we are at the root, no prefix is needed; otherwise, replace directory separators with underscores.
        prefix = "" if rel_path == "." else f"{rel_path.replace(os.sep, '_')}_"
        for fname in filenames:
            if fname.lower().endswith(".png"):
                src_path = os.path.join(dirpath, fname)
                # New filename: prefix + original filename
                new_fname = f"{prefix}{fname}"
                dst_path = os.path.join(dest_folder, new_fname)
                shutil.copy(src_path, dst_path)
                print(f"Copied: {src_path} -> {dst_path}")

# Define input directories
train_input = "/content/KITTI_STEP/frames_split/training/image_02"
test_input  = "/content/KITTI_STEP/frames_split/testing/image_02"

# Define output directories
train_output = "/content/KITTI_STEP/frames_toaugment"  # for training images
test_output  = "/content/KITTI_STEP/frames_real"       # for testing images

# Create output directories if they don't exist
os.makedirs(train_output, exist_ok=True)
os.makedirs(test_output, exist_ok=True)

# Process training and testing directories
print("Processing training images:")
copy_and_rename_recursive(train_input, train_output)

print("\nProcessing testing images:")
copy_and_rename_recursive(test_input, test_output)

print("\n✅ All frames copied with new names.")


Streaming output truncated to the last 5000 lines.
Copied: /content/KITTI_STEP/frames_split/testing/image_02/0019/000312.png -> /content/KITTI_STEP/frames_real/0019_000312.png
Copied: /content/KITTI_STEP/frames_split/testing/image_02/0019/000270.png -> /content/KITTI_STEP/frames_real/0019_000270.png
Copied: /content/KITTI_STEP/frames_split/testing/image_02/0019/000174.png -> /content/KITTI_STEP/frames_real/0019_000174.png
Copied: /content/KITTI_STEP/frames_split/testing/image_02/0019/000322.png -> /content/KITTI_STEP/frames_real/0019_000322.png
Copied: /content/KITTI_STEP/frames_split/testing/image_02/0019/000262.png -> /content/KITTI_STEP/frames_real/0019_000262.png
Copied: /content/KITTI_STEP/frames_split/testing/image_02/0019/000007.png -> /content/KITTI_STEP/frames_real/0019_000007.png
Copied: /content/KITTI_STEP/frames_split/testing/image_02/0019/000388.png -> /content/KITTI_STEP/frames_real/0019_000388.png
Copied: /content/KITTI_STEP/frames_split/testing/image_02/0019/000157.png 

In [6]:
import os
import shutil

# Re-define accessible paths
train_panoptic = "/content/KITTI_STEP/annotations_split/kitti-step/panoptic_maps/train"
val_panoptic = "/content/KITTI_STEP/annotations_split/kitti-step/panoptic_maps/val"
flattened_output = "/content/KITTI_STEP/annotations"

# Create output folder
os.makedirs(flattened_output, exist_ok=True)

# Function to flatten and rename
def flatten_panoptic_maps(src_root):
    for seq in sorted(os.listdir(src_root)):
        seq_path = os.path.join(src_root, seq)
        if os.path.isdir(seq_path):
            for fname in sorted(os.listdir(seq_path)):
                if fname.lower().endswith((".png", ".jpg")):
                    new_name = f"{seq}_{fname}"
                    src_file = os.path.join(seq_path, fname)
                    dst_file = os.path.join(flattened_output, new_name)
                    shutil.copy(src_file, dst_file)

# Apply to both splits
flatten_panoptic_maps(train_panoptic)
flatten_panoptic_maps(val_panoptic)


In [7]:
#Making sure directories sorted
import os

def sorted_dirs(path):
    dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    sorted_dirs = sorted(dirs, key=lambda x: int(x))
    return sorted_dirs

annotations_dir = "/content/KITTI_STEP/annotations"
frames_dir = "/content/KITTI_STEP/frames_real"
frames_diralt = "/content/KITTI_STEP/frames_toaugment"


sorted_annotations = sorted_dirs(annotations_dir)
sorted_frames = sorted_dirs(frames_dir)
sorted_framesalt = sorted_dirs(frames_diralt)

print("✅ Sorted annotation directories:")
print(sorted_annotations)

print("\n✅ Sorted frame directories:")
print(sorted_frames)

print("\n✅ Sorted frame directories:")
print(sorted_framesalt)

✅ Sorted annotation directories:
[]

✅ Sorted frame directories:
[]

✅ Sorted frame directories:
[]


In [9]:
!mkdir -p /content/KITTI_STEP/sign_insertion

In [10]:
!unzip -q /content/traffic_signs.zip -d /content/

In [11]:
!cp -r /content/traffic_signs/* /content/KITTI_STEP/sign_insertion/

In [12]:
import os

# Directory path
sign_dir = "/content/KITTI_STEP/sign_insertion"

# Get a sorted list of all files (filtering for image files)
image_files = sorted([
    f for f in os.listdir(sign_dir)
    if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".webp"))
])

# Rename each file
for idx, fname in enumerate(image_files, start=1):
    ext = os.path.splitext(fname)[1]  # preserve original extension
    new_name = f"sign{idx:02d}{ext}"
    os.rename(os.path.join(sign_dir, fname), os.path.join(sign_dir, new_name))

print("✅ Files renamed successfully.")


✅ Files renamed successfully.


In [ ]:
from PIL import Image
import os

input_dir = "/content/KITTI_STEP/sign_insertion"

for filename in os.listdir(input_dir):
    if filename.endswith(".png"):
        path = os.path.join(input_dir, filename)
        with Image.open(path) as img:
            width, height = img.size
            max_side = max(width, height)

            if max_side > 180:
                scale = 180 / max_side
                new_size = (int(width * scale), int(height * scale))
                resized_img = img.resize(new_size, Image.LANCZOS)
                resized_img.save(path)


In [ ]:
import os

def count_total_kitti_frames(merged_frames_dir="/content/KITTI_STEP/frames"):
    frame_files = [
        f for f in os.listdir(merged_frames_dir)
        if f.endswith(".png") or f.endswith(".jpg")
    ]
    print(f"✅ Total frame files: {len(frame_files)}")

count_total_kitti_frames()


✅ Total frame files: 8008


In [65]:
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np
import os
import random

def inject_phantom_signs_refined(
    frames_dir,
    annotations_dir,
    signs_dir,
    output_allframes_dir,
    output_augmented_dir,
    interval=25,
    duration_range=(3, 5),
    target_classes=[2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18],  # omit road(0), sidewalk(1), sky(10)
    reference_class=7,       # traffic sign
    blur_factor=0.7,         # for fidelity
    max_scale=1.0,           # no up‑scaling beyond this (depth-based)
    max_res=None,            # absolute pixel‑area cap
    max_width=None,          # absolute width cap
    max_height=None,         # absolute height cap
    min_brightness=0.5,      # lower brightness multiplier
    max_brightness=1.3       # upper brightness multiplier
):
    os.makedirs(output_allframes_dir, exist_ok=True)
    os.makedirs(output_augmented_dir,    exist_ok=True)

    # 1) collect and sort frames & panoptic annotations
    frame_files = sorted(f for f in os.listdir(frames_dir) if f.endswith(".png"))
    ann_files   = sorted(f for f in os.listdir(annotations_dir) if f.endswith(".png"))
    assert len(frame_files)==len(ann_files), "Frame/annotation count mismatch"

    # 2) compute average KITTI sign size & set default caps
    ref_w, ref_h, ref_res = [], [], []
    for fn in ann_files:
        pan = np.array(Image.open(os.path.join(annotations_dir, fn)))
        sem = pan[:, :, 2]
        inst= pan[:, :, 0].astype(np.uint16) + 256*pan[:, :, 1].astype(np.uint16)
        for oid in np.unique(inst[sem==reference_class]):
            bbox = get_bbox(inst==oid)
            if bbox:
                w, h = bbox[2], bbox[3]
                ref_w.append(w); ref_h.append(h); ref_res.append(w*h)
    avg_w   = np.mean(ref_w)   if ref_w   else 50
    avg_h   = np.mean(ref_h)   if ref_h   else 50
    avg_res = np.mean(ref_res) if ref_res else avg_w*avg_h

    max_res   = max_res   or (2 * avg_res)
    max_width = max_width or (2 * avg_w)
    max_height= max_height or (2 * avg_h)

    print(f"Avg KITTI sign: {avg_w:.1f}×{avg_h:.1f}px → {int(avg_res)}px²; "
          f"caps: {int(max_res)}px², {int(max_width)}×{int(max_height)}px")

    # 3) load sign PNGs
    signs = []
    for fn in sorted(os.listdir(signs_dir)):
        if fn.lower().endswith(".png"):
            img = Image.open(os.path.join(signs_dir, fn)).convert("RGBA")
            signs.append((img, fn))

    # 4) injection loop
    i = 0
    while i < len(frame_files):
        duration = random.randint(*duration_range)
        if i + duration > len(frame_files):
            break

        # load panoptic for this frame
        pan = np.array(Image.open(os.path.join(annotations_dir, ann_files[i])))
        sem = pan[:, :, 2]
        inst= pan[:, :, 0].astype(np.uint16) + 256*pan[:, :, 1].astype(np.uint16)

        # gather candidate boxes
        candidates = []
        for cls in target_classes:
            for oid in np.unique(inst[sem==cls]):
                bbox = get_bbox(inst==oid)
                if bbox:
                    candidates.append((bbox, cls))
        if not candidates:
            i += interval
            continue

        # pick median-height object
        candidates.sort(key=lambda bc: bc[0][3])
        chosen_box, chosen_cls = candidates[len(candidates)//2]

        # choose random sign
        sign, name = random.choice(signs)
        w0, h0     = sign.size

        # compute all scale factors, keep aspect ratio
        depth_scale = np.clip(chosen_box[3] / avg_h, 0.1, max_scale)
        area_sf     = np.sqrt(max_res / (w0 * h0))
        w_sf        = max_width  / w0
        h_sf        = max_height / h0
        final_scale = min(depth_scale, area_sf, w_sf, h_sf)

        new_w = max(10, int(w0 * final_scale))
        new_h = max(10, int(h0 * final_scale))

        # create mild grainy “projector” effect
        grain_w = max(8, new_w // 2)
        grain_h = max(8, new_h // 2)
        phantom = (
            sign.resize((grain_w, grain_h), Image.BILINEAR)
                .resize((new_w, new_h), Image.BILINEAR)
                .filter(ImageFilter.GaussianBlur(blur_factor))
        )

        print(f"[{frame_files[i]}] → {name}: {new_w}×{new_h}px")

        # paste into each frame of the interval
        for j in range(i, i + duration):
            img = Image.open(os.path.join(frames_dir, frame_files[j])).convert("RGBA")
            pan = np.array(Image.open(os.path.join(annotations_dir, ann_files[j])))
            inst_map = pan[:, :, 0].astype(np.uint16) + 256*pan[:, :, 1].astype(np.uint16)

            if chosen_cls not in np.unique(inst_map):
                continue
            mask_box = get_bbox(inst_map == chosen_cls)
            if not mask_box:
                continue

            x, y, w_box, h_box = mask_box
            px = int(x + w_box/2 - phantom.width/2)
            py = int(y - phantom.height)
            px = int(np.clip(px, 0, img.width  - phantom.width))
            py = int(np.clip(py, 0, img.height - phantom.height))

            # brightness match & cap
            patch = img.crop((px, py, px + phantom.width, py + phantom.height)).convert("RGB")
            bf = np.mean(np.array(patch)) / 127.5
            bf = float(np.clip(bf, min_brightness, max_brightness))
            adj = ImageEnhance.Brightness(phantom).enhance(bf)

            comp = img.copy()
            comp.paste(adj, (px, py), adj)

            comp.convert("RGB").save(os.path.join(output_allframes_dir, frame_files[j]))
            comp.convert("RGB").save(os.path.join(output_augmented_dir,  frame_files[j]))

        i += interval

def get_bbox(mask):
    """Return (x, y, w, h) of binary mask or None"""
    coords = np.argwhere(mask)
    if coords.size == 0:
        return None
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0)
    return int(x0), int(y0), int(x1 - x0), int(y1 - y0)


# ——— example call ———
inject_phantom_signs_refined(
    frames_dir           = "/content/KITTI_STEP/frames_toaugment",
    annotations_dir      = "/content/KITTI_STEP/annotations",
    signs_dir            = "/content/KITTI_STEP/sign_insertion",
    output_allframes_dir = "/content/KITTI_STEP/output_allaugmented",
    output_augmented_dir = "/content/KITTI_STEP/output_augmented",
    interval             = 25,
    duration_range       = (3, 5),
    target_classes       = [2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18],
    reference_class      = 7,
    blur_factor          = 0.7,
    max_scale            = 0.03,
    max_res              = None,  # default = 2×avg_area
    max_width            = None,  # default = 2×avg_w
    max_height           = None,  # default = 2×avg_h
    min_brightness       = 0.7,
    max_brightness       = 1.0
)


Avg KITTI sign: 724.9×159.8px → 122114px²; caps: 244229px², 1449×319px
[0000_000000.png] → Vienna_Convention_road_sign_C6-V1.svg.png: 38×38px
[0000_000125.png] → Vienna_Convention_road_sign_C3b-V3-1.svg.png: 38×38px
[0000_000150.png] → 1920px-Vienna_Convention_road_sign_Aa-15a-V1.svg.png: 57×50px
[0001_000021.png] → Vienna_Convention_road_sign_C6-V1.svg.png: 38×38px
[0001_000046.png] → Vienna_Convention_road_sign_B2a.svg.png: 38×38px
[0001_000071.png] → Vienna_Convention_road_sign_C19.svg.png: 38×38px
[0002_000049.png] → 1920px-Vienna_Convention_road_sign_Aa-4a-V1.svg.png: 57×50px
[0002_000074.png] → Vienna_Convention_road_sign_D2a-V2.svg.png: 38×38px
[0002_000099.png] → 1280px-Vienna_Convention_road_sign_C3a-V1-1.svg.png: 38×38px
[0002_000124.png] → Vienna_Convention_road_sign_E3a.svg.png: 38×38px
[0002_000149.png] → Vienna_Convention_road_sign_Aa-17a-V1.svg.png: 57×51px
[0002_000174.png] → Vienna_Convention_road_sign_C3b-V3-1.svg.png: 38×38px
[0002_000199.png] → Vienna_Convention_roa

In [64]:
import os
import shutil

# List your output directories here
output_dirs = [
    "/content/KITTI_STEP/output_allaugmented",
    "/content/KITTI_STEP/output_augmented"
]

for d in output_dirs:
    if os.path.isdir(d):
        for entry in os.listdir(d):
            path = os.path.join(d, entry)
            try:
                if os.path.isdir(path):
                    shutil.rmtree(path)
                else:
                    os.remove(path)
            except Exception as e:
                print(f"Failed to remove {path}: {e}")
        print(f"✅ Cleared directory: {d}")
    else:
        print(f"⚠️  Directory does not exist: {d}")


✅ Cleared directory: /content/KITTI_STEP/output_allaugmented
✅ Cleared directory: /content/KITTI_STEP/output_augmented


In [ ]:
!zip -r /content/KITTI_STEP_PACKAGE.zip /content/KITTI_STEP/output_augmentedframes/
!zip -r /content/KITTI_STEP_PACKAGE.zip /content/KITTI_STEP/output_allframes/
!zip -r /content/KITTI_STEP_PACKAGE.zip /content/KITTI_STEP/frames/

Streaming output truncated to the last 5000 lines.
  adding: content/KITTI_STEP/frames/0009_000030.png (deflated 1%)
  adding: content/KITTI_STEP/frames/0007_000106.png (deflated 1%)
  adding: content/KITTI_STEP/frames/0019_000939.png (deflated 2%)
  adding: content/KITTI_STEP/frames/0011_000081.png (deflated 1%)
  adding: content/KITTI_STEP/frames/0013_000106.png (deflated 2%)
  adding: content/KITTI_STEP/frames/0000_000042.png (deflated 2%)
  adding: content/KITTI_STEP/frames/0008_000301.png (deflated 7%)
  adding: content/KITTI_STEP/frames/0007_000182.png (deflated 8%)
  adding: content/KITTI_STEP/frames/0007_000091.png (deflated 2%)
  adding: content/KITTI_STEP/frames/0019_000559.png (deflated 2%)
  adding: content/KITTI_STEP/frames/0009_000555.png (deflated 0%)
  adding: content/KITTI_STEP/frames/0011_000239.png (deflated 1%)
  adding: content/KITTI_STEP/frames/0013_000112.png (deflated 2%)
  adding: content/KITTI_STEP/frames/0007_000035.png (deflated 2%)
  adding: content/KITTI_S

In [66]:
import os
import shutil

def fill_missing_frames(
    source_dir="/content/KITTI_STEP/frames_toaugment",
    target_dir="/content/KITTI_STEP/output_allaugmented"
):
    os.makedirs(target_dir, exist_ok=True)

    source_frames = sorted([f for f in os.listdir(source_dir) if f.endswith(".png")])
    target_frames = set(os.listdir(target_dir))

    filled_count = 0
    for fname in source_frames:
        if fname not in target_frames:
            src_path = os.path.join(source_dir, fname)
            tgt_path = os.path.join(target_dir, fname)
            shutil.copy2(src_path, tgt_path)
            filled_count += 1
            print(f"🟢 Copied {fname} to output_allframes")

    print(f"\n✅ Done. Filled {filled_count} missing frames.")

# Run the function
fill_missing_frames()


Streaming output truncated to the last 5000 lines.
🟢 Copied 0008_000036.png to output_allframes
🟢 Copied 0008_000037.png to output_allframes
🟢 Copied 0008_000038.png to output_allframes
🟢 Copied 0008_000039.png to output_allframes
🟢 Copied 0008_000040.png to output_allframes
🟢 Copied 0008_000046.png to output_allframes
🟢 Copied 0008_000047.png to output_allframes
🟢 Copied 0008_000048.png to output_allframes
🟢 Copied 0008_000049.png to output_allframes
🟢 Copied 0008_000050.png to output_allframes
🟢 Copied 0008_000051.png to output_allframes
🟢 Copied 0008_000052.png to output_allframes
🟢 Copied 0008_000053.png to output_allframes
🟢 Copied 0008_000054.png to output_allframes
🟢 Copied 0008_000055.png to output_allframes
🟢 Copied 0008_000056.png to output_allframes
🟢 Copied 0008_000057.png to output_allframes
🟢 Copied 0008_000058.png to output_allframes
🟢 Copied 0008_000059.png to output_allframes
🟢 Copied 0008_000060.png to output_allframes
🟢 Copied 0008_000061.png to output_allframes
🟢 Co

In [69]:
!zip -r /content/KITTI_augmented.zip /content/KITTI_STEP/output_allaugmented

Streaming output truncated to the last 5000 lines.
  adding: content/KITTI_STEP/output_allaugmented/0009_000030.png (deflated 1%)
  adding: content/KITTI_STEP/output_allaugmented/0007_000106.png (deflated 1%)
  adding: content/KITTI_STEP/output_allaugmented/0019_000939.png (deflated 2%)
  adding: content/KITTI_STEP/output_allaugmented/0011_000081.png (deflated 0%)
  adding: content/KITTI_STEP/output_allaugmented/0013_000106.png (deflated 2%)
  adding: content/KITTI_STEP/output_allaugmented/0000_000042.png (deflated 2%)
  adding: content/KITTI_STEP/output_allaugmented/0008_000301.png (deflated 7%)
  adding: content/KITTI_STEP/output_allaugmented/0007_000182.png (deflated 8%)
  adding: content/KITTI_STEP/output_allaugmented/0007_000091.png (deflated 2%)
  adding: content/KITTI_STEP/output_allaugmented/0019_000559.png (deflated 2%)
  adding: content/KITTI_STEP/output_allaugmented/0009_000555.png (deflated 0%)
  adding: content/KITTI_STEP/output_allaugmented/0011_000239.png (deflated 1%)
 

In [70]:
from google.colab import files
files.download('/content/KITTI_augmented.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:
import cv2, os
from collections import defaultdict

def create_videos_with_cv2(frames_dir, output_dir, fps=10, codec="FFV1"):
    os.makedirs(output_dir, exist_ok=True)

    grouped = defaultdict(list)
    for fname in sorted(os.listdir(frames_dir)):
        if fname.endswith(".png"):
            prefix = fname[:4]
            grouped[prefix].append(fname)

    for prefix, frame_list in grouped.items():
        frame_list.sort()
        first = os.path.join(frames_dir, frame_list[0])
        img = cv2.imread(first)
        if img is None:
            print(f"❌ Skipping {prefix}: cannot read {first}")
            continue
        h, w = img.shape[:2]

        out_path = os.path.join(output_dir, f"{prefix}.avi")
        print(f"🎞️  {prefix}: {len(frame_list)} frames → {out_path}")

        fourcc = cv2.VideoWriter_fourcc(*codec)
        writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
        if not writer.isOpened():
            raise RuntimeError(f"Failed to open writer for {out_path}")

        for fname in frame_list:
            frame = cv2.imread(os.path.join(frames_dir, fname))
            if frame is None:
                print(f"⚠️  Missing/bad frame: {fname} – inserting blank placeholder")
                frame = 0 * img  # black frame
            writer.write(frame)

        writer.release()
        print(f"✅ Saved {out_path}\n")


In [73]:
!mkdir -p /content/KITTI_STEP/augmented_videos
create_videos_with_cv2(
    frames_dir="/content/KITTI_STEP/output_allaugmented",
    output_dir="/content/KITTI_STEP/augmented_videos",
    fps=10,
    codec="FFV1"  # you can also try "avc1" if needed by the extractor
)

🎞️  0000: 154 frames → /content/KITTI_STEP/augmented_videos/0000.avi
✅ Saved /content/KITTI_STEP/augmented_videos/0000.avi

🎞️  0001: 447 frames → /content/KITTI_STEP/augmented_videos/0001.avi
✅ Saved /content/KITTI_STEP/augmented_videos/0001.avi

🎞️  0002: 233 frames → /content/KITTI_STEP/augmented_videos/0002.avi
✅ Saved /content/KITTI_STEP/augmented_videos/0002.avi

🎞️  0003: 144 frames → /content/KITTI_STEP/augmented_videos/0003.avi
✅ Saved /content/KITTI_STEP/augmented_videos/0003.avi

🎞️  0004: 314 frames → /content/KITTI_STEP/augmented_videos/0004.avi
✅ Saved /content/KITTI_STEP/augmented_videos/0004.avi

🎞️  0005: 297 frames → /content/KITTI_STEP/augmented_videos/0005.avi
✅ Saved /content/KITTI_STEP/augmented_videos/0005.avi

🎞️  0006: 270 frames → /content/KITTI_STEP/augmented_videos/0006.avi
✅ Saved /content/KITTI_STEP/augmented_videos/0006.avi

🎞️  0007: 800 frames → /content/KITTI_STEP/augmented_videos/0007.avi
✅ Saved /content/KITTI_STEP/augmented_videos/0007.avi

🎞️  0008

In [75]:
!mkdir -p /content/KITTI_STEP/real_videos
create_videos_with_cv2(
    frames_dir="/content/KITTI_STEP/frames_real",
    output_dir="/content/KITTI_STEP/real_videos",
    fps=10,
    codec="FFV1"  # you can also try "avc1" if needed by the extractor
)

🎞️  0000: 465 frames → /content/KITTI_STEP/real_videos/0000.avi
✅ Saved /content/KITTI_STEP/real_videos/0000.avi

🎞️  0001: 147 frames → /content/KITTI_STEP/real_videos/0001.avi
✅ Saved /content/KITTI_STEP/real_videos/0001.avi

🎞️  0002: 243 frames → /content/KITTI_STEP/real_videos/0002.avi
✅ Saved /content/KITTI_STEP/real_videos/0002.avi

🎞️  0003: 257 frames → /content/KITTI_STEP/real_videos/0003.avi
✅ Saved /content/KITTI_STEP/real_videos/0003.avi

🎞️  0004: 421 frames → /content/KITTI_STEP/real_videos/0004.avi
✅ Saved /content/KITTI_STEP/real_videos/0004.avi

🎞️  0005: 809 frames → /content/KITTI_STEP/real_videos/0005.avi
✅ Saved /content/KITTI_STEP/real_videos/0005.avi

🎞️  0006: 114 frames → /content/KITTI_STEP/real_videos/0006.avi
✅ Saved /content/KITTI_STEP/real_videos/0006.avi

🎞️  0007: 215 frames → /content/KITTI_STEP/real_videos/0007.avi
✅ Saved /content/KITTI_STEP/real_videos/0007.avi

🎞️  0008: 165 frames → /content/KITTI_STEP/real_videos/0008.avi
✅ Saved /content/KITTI_S

In [82]:
import cv2
import os

def print_avi_resolutions(frames_dir):
    for fname in sorted(os.listdir(frames_dir)):
        if not fname.lower().endswith(".avi"):
            continue
        path = os.path.join(frames_dir, fname)
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"❌ Cannot open {fname}")
            continue
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print(f"{fname} → {w}×{h}")
        cap.release()

# Example usage:
print_avi_resolutions("/content/KITTI_STEP/real_videos")


0000.avi → 1242×374
0001.avi → 1242×374
0002.avi → 1242×374
0003.avi → 1242×374
0004.avi → 1242×374
0005.avi → 1242×374
0006.avi → 1242×374
0007.avi → 1242×374
0008.avi → 1242×374
0009.avi → 1242×374
0010.avi → 1242×374
0011.avi → 1242×374
0012.avi → 1242×374
0013.avi → 1242×374
0014.avi → 1242×374
0015.avi → 1242×374
0016.avi → 1242×374
0017.avi → 1224×370
0018.avi → 1224×370
0019.avi → 1224×370
0020.avi → 1224×370
0021.avi → 1224×370
0022.avi → 1224×370
0023.avi → 1224×370
0024.avi → 1224×370
0025.avi → 1224×370
0026.avi → 1224×370
0027.avi → 1226×370
0028.avi → 1226×370


In [81]:
import os
import subprocess

# ── CONFIG ────────────────────────────────────────────────────────────────
input_dir  = "/content/KITTI_STEP/augmented_videos"    # your source AVIs
output_dir = "/content/KITTI_STEP/fixed_augmented_videos"  # where to write resized AVIs
target_w, target_h = 1242, 374                     # canonical resolution

# ── PREP ─────────────────────────────────────────────────────────────────
os.makedirs(output_dir, exist_ok=True)

# ── PROCESS LOOP ──────────────────────────────────────────────────────────
for fname in sorted(os.listdir(input_dir)):
    if not fname.lower().endswith(".avi"):
        continue
    in_path  = os.path.join(input_dir, fname)
    # append size suffix to avoid clobbering
    base, _   = os.path.splitext(fname)
    out_fname = f"{base}_{target_w}x{target_h}.avi"
    out_path  = os.path.join(output_dir, out_fname)

    print(f"Resizing & re‑encoding → {out_fname}")
    cmd = [
        "ffmpeg",
        "-y",                     # overwrite if exists
        "-i", in_path,            # input file
        "-vf", f"scale={target_w}:{target_h}",  # force scale
        "-c:v", "ffv1",           # FFV1 codec
        out_path
    ]
    subprocess.run(cmd, check=True)

print("✅ All done! Resized AVIs are in:", output_dir)


Resizing & re‑encoding → 0000_1242x374.avi
Resizing & re‑encoding → 0001_1242x374.avi
Resizing & re‑encoding → 0002_1242x374.avi
Resizing & re‑encoding → 0003_1242x374.avi
Resizing & re‑encoding → 0004_1242x374.avi
Resizing & re‑encoding → 0005_1242x374.avi
Resizing & re‑encoding → 0006_1242x374.avi
Resizing & re‑encoding → 0007_1242x374.avi
Resizing & re‑encoding → 0008_1242x374.avi
Resizing & re‑encoding → 0009_1242x374.avi
Resizing & re‑encoding → 0010_1242x374.avi
Resizing & re‑encoding → 0011_1242x374.avi
Resizing & re‑encoding → 0012_1242x374.avi
Resizing & re‑encoding → 0013_1242x374.avi
Resizing & re‑encoding → 0014_1242x374.avi
Resizing & re‑encoding → 0015_1242x374.avi
Resizing & re‑encoding → 0016_1242x374.avi
Resizing & re‑encoding → 0017_1242x374.avi
Resizing & re‑encoding → 0018_1242x374.avi
Resizing & re‑encoding → 0019_1242x374.avi
Resizing & re‑encoding → 0020_1242x374.avi
✅ All done! Resized AVIs are in: /content/KITTI_STEP/fixed_augmented_videos


In [83]:
import os
import subprocess
import cv2

# ── CONFIG ──────────────────────────────────────────────────────────────
INPUT_DIR  = "/content/KITTI_STEP/real_videos"         # where your .avi files live
OUTPUT_DIR = "/content/KITTI_STEP/fixed_real_videos"     # where resized avis go
TARGET_W, TARGET_H = 1242, 374                         # your canonical resolution
# ─────────────────────────────────────────────────────────────────────────

os.makedirs(OUTPUT_DIR, exist_ok=True)

for fname in sorted(os.listdir(INPUT_DIR)):
    if not fname.lower().endswith(".avi"):
        continue

    in_path = os.path.join(INPUT_DIR, fname)
    cap = cv2.VideoCapture(in_path)
    if not cap.isOpened():
        print(f"❌ Could not open {fname}, skipping")
        continue

    # read declared width / height
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    if (w, h) == (TARGET_W, TARGET_H):
        print(f"✅ Skipping {fname}: already {w}×{h}")
        continue

    # build output path
    base, _ = os.path.splitext(fname)
    out_name = f"{base}_{TARGET_W}x{TARGET_H}.avi"
    out_path = os.path.join(OUTPUT_DIR, out_name)

    print(f"🔄 Converting {fname}: {w}×{h} → {TARGET_W}×{TARGET_H}")
    subprocess.run([
        "ffmpeg", "-y",
        "-i", in_path,
        "-c:v", "ffv1",
        "-vf", f"scale={TARGET_W}:{TARGET_H}",
        out_path
    ], check=True)

print("🎉 All done.")


✅ Skipping 0000.avi: already 1242×374
✅ Skipping 0001.avi: already 1242×374
✅ Skipping 0002.avi: already 1242×374
✅ Skipping 0003.avi: already 1242×374
✅ Skipping 0004.avi: already 1242×374
✅ Skipping 0005.avi: already 1242×374
✅ Skipping 0006.avi: already 1242×374
✅ Skipping 0007.avi: already 1242×374
✅ Skipping 0008.avi: already 1242×374
✅ Skipping 0009.avi: already 1242×374
✅ Skipping 0010.avi: already 1242×374
✅ Skipping 0011.avi: already 1242×374
✅ Skipping 0012.avi: already 1242×374
✅ Skipping 0013.avi: already 1242×374
✅ Skipping 0014.avi: already 1242×374
✅ Skipping 0015.avi: already 1242×374
✅ Skipping 0016.avi: already 1242×374
🔄 Converting 0017.avi: 1224×370 → 1242×374
🔄 Converting 0018.avi: 1224×370 → 1242×374
🔄 Converting 0019.avi: 1224×370 → 1242×374
🔄 Converting 0020.avi: 1224×370 → 1242×374
🔄 Converting 0021.avi: 1224×370 → 1242×374
🔄 Converting 0022.avi: 1224×370 → 1242×374
🔄 Converting 0023.avi: 1224×370 → 1242×374
🔄 Converting 0024.avi: 1224×370 → 1242×374
🔄 Converti

In [84]:
!zip -r /content/fixed_real_videos.zip /content/KITTI_STEP/fixed_real_videos

  adding: content/KITTI_STEP/fixed_real_videos/ (stored 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0028_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0020_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0023_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0018_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0027_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0026_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0024_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0019_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0017_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0021_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0022_1242x374.avi (deflated 0%)
  adding: content/KITTI_STEP/fixed_real_videos/0025_1242x374.avi (defla

In [85]:
from google.colab import files
files.download('/content/fixed_real_videos.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
%%bash
cd /content
git clone https://github.com/tensorflow/models.git


Cloning into 'models'...
remote: Enumerating objects: 99739, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 99739 (delta 68), reused 53 (delta 46), pack-reused 99641 (from 3)
Receiving objects: 100% (99739/99739), 626.61 MiB | 29.41 MiB/s, done.
Resolving deltas: 100% (72589/72589), done.
/content/models/research


In [3]:
# Install the protobuf compiler (if not already installed on Colab).
!apt-get -qq install protobuf-compiler python-pil python-lxml

# Compile the proto files in the object_detection folder.
!protoc object_detection/protos/*.proto --python_out=.


E: Unable to locate package python-pil
E: Unable to locate package python-lxml


In [2]:
!pip install tensorflow-object-detection-api


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.0/739.0 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.8 MB/s eta 0:00:00
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844488 sha256=ac855b057cae7cbf034f4046577e20

In [3]:
!pip install --upgrade pip
!pip install protobuf==3.20.3  # fixes “Descriptors cannot be created directly” errors :contentReference[oaicite:0]{index=0}


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.0 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you ha

In [2]:
from object_detection.utils import label_map_util


In [4]:
import glob
import os
os.chdir('/content/GhostBustersRevised')
# adjust the glob to match your video files
for vid in glob.glob('/content/KITTI_STEP/fixed_augmented_videos/*.avi'):
    print("Processing", vid)
    !python /content/GhostBustersRevised/GhostBusters.py -pf "{vid}"


Processing /content/KITTI_STEP/fixed_augmented_videos/0016_1242x374.avi
2025-04-17 00:06:17.210276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744848377.244093  118043 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744848377.254366  118043 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744848377.278958  118043 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744848377.278999  118043 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:174

In [6]:
import glob
import os
os.chdir('/content/GhostBustersRevised')
# adjust the glob to match your video files
for vid in glob.glob('/content/KITTI_STEP/fixed_real_videos/*.avi'):
    print("Processing", vid)
    !python /content/GhostBustersRevised/GhostBusters.py -pr "{vid}"

Processing /content/KITTI_STEP/fixed_real_videos/0028_1242x374.avi
2025-04-17 02:06:35.846902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744855595.867925  149052 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744855595.874245  149052 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744855595.891703  149052 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744855595.891735  149052 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17448555

In [7]:
os.chdir('/content/GhostBustersRevised')
!python GhostBusters.py -t --exp_epochs 25 --com_epochs 25


2025-04-17 02:48:41.205559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744858121.226745  160268 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744858121.233110  160268 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744858121.250550  160268 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744858121.250577  160268 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744858121.250581  160268 computation_placer.cc:177] computation placer alr

In [5]:
!zip -r /content/fake_data.zip /content/GhostBustersRevised/data/fake/


Streaming output truncated to the last 5000 lines.
  adding: content/GhostBustersRevised/data/fake/light/155543_27_l.npy (deflated 84%)
  adding: content/GhostBustersRevised/data/fake/light/384815_135_l.npy (deflated 82%)
  adding: content/GhostBustersRevised/data/fake/light/376335_35_l.npy (deflated 83%)
  adding: content/GhostBustersRevised/data/fake/light/384815_67_l.npy (deflated 93%)
  adding: content/GhostBustersRevised/data/fake/light/750331_23_l.npy (deflated 88%)
  adding: content/GhostBustersRevised/data/fake/light/576912_29_l.npy (deflated 89%)
  adding: content/GhostBustersRevised/data/fake/light/626124_30_l.npy (deflated 86%)
  adding: content/GhostBustersRevised/data/fake/light/155543_46_l.npy (deflated 84%)
  adding: content/GhostBustersRevised/data/fake/light/626124_144_l.npy (deflated 90%)
  adding: content/GhostBustersRevised/data/fake/light/217292_24_l.npy (deflated 92%)
  adding: content/GhostBustersRevised/data/fake/light/580691_21_l.npy (deflated 89%)
  adding: co

In [ ]:
!unzip -q /content/faster_rcnn_inception_resnet_v2_atrous.zip -d /content/GhostBusters/models/faster_rcnn_inception_resnet_v2_atrous

In [4]:
# Remove if previously half-installed
!rm -rf /content/tf_models

# Clone TensorFlow models repo to a safe location
!git clone --depth 1 https://github.com/tensorflow/models /content/tf_models

# Compile protobufs
!apt-get install -y protobuf-compiler
!cd /content/tf_models/research && protoc object_detection/protos/*.proto --python_out=.

# Install the Object Detection API
!cp /content/tf_models/research/object_detection/packages/tf2/setup.py /content/tf_models/research/
!pip install -e /content/tf_models/research/

# Test install
!python /content/tf_models/research/object_detection/builders/model_builder_tf2_test.py


Cloning into '/content/tf_models'...
remote: Enumerating objects: 4327, done.
remote: Counting objects: 100% (4327/4327), done.
remote: Compressing objects: 100% (3345/3345), done.
remote: Total 4327 (delta 1208), reused 2052 (delta 910), pack-reused 0 (from 0)
Receiving objects: 100% (4327/4327), 53.65 MiB | 20.53 MiB/s, done.
Resolving deltas: 100% (1208/1208), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Obtaining file:///content/tf_models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 7.9 M

2025-04-16 23:50:11.160390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744847411.191575  113426 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744847411.201320  113426 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744847411.227269  113426 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744847411.227311  113426 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744847411.227319  113426 computation_placer.cc:177] computation placer alr

In [86]:
!pip install protobuf==3.20.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.0 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
import os

base_path = "/content/gb_features"
subdirs = ["context", "light", "surface", "optical"]

# Create main and subdirectories
os.makedirs(base_path, exist_ok=True)
for sub in subdirs:
    os.makedirs(os.path.join(base_path, sub), exist_ok=True)

print(f"✅ Created: {base_path} with subdirectories {subdirs}")


✅ Created: /content/gb_features with subdirectories ['context', 'light', 'surface', 'optical']


In [ ]:
import os

base_path = "/content/GhostBusters/data"
folders_to_clear = ["real", "fake", "real_nosign"]

for folder in folders_to_clear:
    folder_path = os.path.join(base_path, folder)
    if os.path.exists(folder_path):
        for f in os.listdir(folder_path):
            file_path = os.path.join(folder_path, f)
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                # Recursively delete subdirectories
                import shutil
                shutil.rmtree(file_path)
        print(f"✅ Cleared: {folder_path}")
    else:
        print(f"⚠️ Folder not found: {folder_path}")


✅ Cleared: /content/GhostBusters/data/real
✅ Cleared: /content/GhostBusters/data/fake
✅ Cleared: /content/GhostBusters/data/real_nosign


In [ ]:
import os
import random

os.chdir("/content/GhostBusters")

# Path to videos
video_dir = "/content/merged_videos"
video_files = sorted([os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith(".mp4")])

# Shuffle and split into two halves
random.seed(42)
random.shuffle(video_files)
split_index = len(video_files) // 2
train_videos = video_files[:split_index]
val_videos = video_files[split_index:]

# Process half the videos as REAL examples
for vid_path in train_videos:
    print(f"Processing real/real_nosign: {vid_path}")
    !python /content/GhostBusters/GhostBusters.py -pr "{vid_path}"

# Optionally: process fake from other videos (or use -pf on same)
# for vid_path in train_videos:
#     print(f"Processing fake: {vid_path}")
#     !python GhostBusters.py -pf "{vid_path}"

# ✅ Remaining videos (val_videos) are left untouched for validation
print(f"✅ Processed {len(train_videos)} videos for training. {len(val_videos)} held out.")


Processing real/real_nosign: /content/merged_videos/0012.mp4
2025-04-15 00:12:02.350725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744675922.371682  126180 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744675922.378003  126180 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744675922.394535  126180 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744675922.394561  126180 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744675922.394

In [ ]:
import os

# Make sure you're in the correct directory for relative model path to work
os.chdir("/content/GhostBusters")

# Loop through training videos and extract features for fake signs
for vid_path in train_videos:
    print(f"Processing fake: {vid_path}")
    !python GhostBusters.py -pf "{vid_path}"


Processing fake: /content/merged_videos/0012.mp4
2025-04-15 00:59:30.435519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744678770.456812  138844 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744678770.463313  138844 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744678770.480963  138844 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744678770.480994  138844 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744678770.481005  138844 

In [ ]:
!unzip -q /content/GhostBusters_four.zip -d /

In [ ]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version (imported):", keras.__version__)


TensorFlow version: 2.18.0
Keras version (imported): 3.8.0


In [ ]:
import os

# Make sure you're in the correct directory for relative model path to work
os.chdir("/content/GhostBusters")

!python GhostBusters.py -t


2025-04-15 03:25:42.983614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744687543.017725   13924 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744687543.028005   13924 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 03:25:43.062936: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loading files from disk: data/real/context/101711_100_c.npy ...
Loading files from disk: data/real/context/631248_59_c.npy ..